In [1]:
import spectrum
import pike as pk
import pike_parallel as parall
import csv
import pandas as pd
from glob import glob
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import PairwiseKernel
import random
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pymzml
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.metrics import accuracy_score


In [16]:
def varianceStabilizer(spectrumObject):
    '''Pre-processing function for manipulating intensities.
    Commonly performed to stabilize their variance
    Parameters
    ----------
    SpectrumObject
    
    Returns
    -------
    SpectrumObject with stabilized variance.
    '''
    stabilizer = spectrum.VarStabilizer()
    return stabilizer(spectrumObject)

def baselineRemove( spectrumObject):
    '''Pre-processing function for baseline correction (also referred to as background removal).
    Parameters
    ----------
    SpectrumObject
    
    Returns
    -------
    SpectrumObject.
    '''
    remover = spectrum.BaselineCorrecter(method="SNIP")
    return remover(spectrumObject)

def intensityCalibration( spectrumObject):
    '''Pre-processing function for normalizing the intensity of a spectrum.
    Commonly referred to as total ion current (TIC) calibration.

    Parameters
    ----------
    SpectrumObject
    
    Returns
    -------
    SpectrumObject.
    '''

    calibration = spectrum.Normalizer()
    return calibration(spectrumObject)

def smoother( spectrumObject):
    '''Pre-processing function for smoothing. Uses Savitzky-Golay filter.

    Parameters
    ----------
    SpectrumObject

    Returns
    -------
    SpectrumObject
    '''
    smoother =  spectrum.Smoother()
    return smoother(spectrumObject)

def trimming(spectrumObject):
    '''Pre-processing function for trimming ends of a spectrum.
    This can be used to remove inaccurate measurements.

    Parameters
    ----------
    SpectrumObject
    '''

    trimmer = spectrum.Trimmer()
    return trimmer(spectrumObject)


def binner(spectrumObject):
    '''Pre-processing function for binning spectra in equal-width bins.

    Parameters
    ----------
    spectrumObject
    '''
    binner = spectrum.Binner()
    return binner(spectrumObject)
spectra = spectrum.SpectrumObject()

In [17]:
# Datos del Gregorio sin extracción de proteínas
semanas = ['Semana 1', 'Semana 2', 'Semana 3']
clases = ['RT023', 'RT027', 'RT078', 'RT106', 'RT165', 'RT181']
medios = ['Medio Ch', 'Medio Br', 'Medio Cl', 'Medio Sc', 'GU']

dic = {}
Y_train = []
baseline = []

for medio in medios:
    for semana in semanas:
        samples = []
        for clase in clases:
            
            ruta = f'Z:/bacteria_id/C_diff/Reproducibilidad/ClostiRepro/ClostriRepro/Reproducibilidad No extracción/{medio}/{semana}/{clase}' 
            
            if os.path.exists(ruta):
                for f in os.listdir(ruta):
                    ruta_f = os.path.join(ruta, f)
                    
                    if medio == 'Medio Ch'and semana == 'Semana 1':
                        # Si el archivo es un .mzml
                        if 'mzml' in ruta_f:
                            run = pymzml.run.Reader(ruta_f)
                            spectro = run[0]
                            s = spectrum.SpectrumObject(mz=spectro.mz, intensity=spectro.i)
                            baseline.append(s)
                            Y_train.append(clase)
                        else: 
                            carpetas = [subf for subf in os.listdir(ruta_f)]
                            if carpetas:
                                ruta_f = os.path.join(ruta_f, carpetas[0])
                                # Buscar archivos 'fid' y 'acqu' en las subcarpetas
                                fid_files = glob(os.path.join(ruta_f, '*', '1SLin', 'fid'))
                                acqu_files = glob(os.path.join(ruta_f, '*', '1SLin', 'acqu'))

                                ruta_fid = fid_files[0]
                                ruta_acqu = acqu_files[0]
                                s = spectra.from_bruker(ruta_acqu, ruta_fid)
                                baseline.append(s)
                                Y_train.append(clase)       
                    
                    else:
                        # Si el archivo es un .mzml
                        if 'mzml' in ruta_f:
                            run = pymzml.run.Reader(ruta_f)
                            
                            spectro = [r for r in run]
                            s = spectrum.SpectrumObject(mz=spectro[0].mz, intensity=spectro[0].i)
                            samples.append(s)
                                    
                                    
                        else: 
                            carpetas = [subf for subf in os.listdir(ruta_f)]
                            
                            ruta_f = os.path.join(ruta, f, carpetas[0])
                            fid_files = glob(os.path.join(ruta_f, '*', '1SLin', 'fid'))
                            acqu_files = glob(os.path.join(ruta_f, '*', '1SLin', 'acqu'))

                            ruta_fid = fid_files[0]
                            ruta_acqu = acqu_files[0]
                            s = spectra.from_bruker(ruta_acqu, ruta_fid)
                            samples.append(s) 
                            

            dic[f'{semana}_{medio}'] = samples


In [14]:
# Datos del Gregorio con extracción de proteínas

clases = ['RT023', 'RT027', 'RT078', 'RT106', 'RT165', 'RT181']
medios = ['Chx', 'Brx', 'Clx', 'Scx', 'GU']

dic_extra = {}


for medio in medios:
        samples = []
        for clase in clases:
            
            ruta = f'Z:/bacteria_id/C_diff/Reproducibilidad/ClostiRepro/ClostriRepro/Reproducibilidad Extracción/{medio}/{clase}' 
            
            if os.path.exists(ruta):
                for f in os.listdir(ruta):
                    ruta_f = os.path.join(ruta, f)
                    
                        
                    if 'mzml' in ruta_f:
                        run = pymzml.run.Reader(ruta_f)
                        
                        spectro = [r for r in run]
                        s = spectrum.SpectrumObject(mz=spectro[0].mz, intensity=spectro[0].i)
                        samples.append(s)
                                
                                
                    else: 
                        carpetas = [subf for subf in os.listdir(ruta_f)]
                        
                        ruta_f = os.path.join(ruta, f, carpetas[0])
                        fid_files = glob(os.path.join(ruta_f, '*', '1SLin', 'fid'))
                        acqu_files = glob(os.path.join(ruta_f, '*', '1SLin', 'acqu'))

                        ruta_fid = fid_files[0]
                        ruta_acqu = acqu_files[0]
                        s = spectra.from_bruker(ruta_acqu, ruta_fid)
                        samples.append(s) 
                            

            dic_extra[f'{medio}'] = samples


KeyboardInterrupt: 

In [18]:
#preprocesado baseline
spec1C_mz = []
spec1C_intensity = []

for i, s in enumerate(baseline):
    
    # Preprocessing: Varaince stab.,smoother, baseline removal, intensity caibration (TIC), trimming
    sO_s1 = varianceStabilizer(s)
    sO_s1 = smoother(sO_s1)
    sO_s1 = baselineRemove(sO_s1)
    sO_s1 = intensityCalibration(sO_s1)
    sO_s1 = trimming(sO_s1)
    # Binning
    sO_s1 = binner(sO_s1)

    spec1C_mz.append(np.array(sO_s1.mz))
    spec1C_intensity.append(np.array(sO_s1.intensity))


spec1C_mz = np.array(spec1C_mz)
spec1C_intensity = np.array(spec1C_intensity)


In [5]:
#preprocesado test samples
specTest_mz = []
specTest_intensity = []
test = dic['Semana 1_GU']
#test = dic_extra['Brx']
for i, s in enumerate(test):
    
    
    # Preprocessing: Varaince stab.,smoother, baseline removal, intensity caibration (TIC), trimming
    sO_s1 = varianceStabilizer(s)
    sO_s1 = smoother(sO_s1)
    sO_s1 = baselineRemove(sO_s1)
    sO_s1 = intensityCalibration(sO_s1)
    sO_s1 = trimming(sO_s1)
    # Binning
    sO_s1 = binner(sO_s1)

    specTest_mz.append(np.array(sO_s1.mz))
    specTest_intensity.append(np.array(sO_s1.intensity))


specTest_mz = np.array(specTest_mz)
specTest_intensity = np.array(specTest_intensity)

# RF

In [7]:
n_estimators = [10, 50, 100, 200, 300]

## baseline

In [8]:

param_grid = {'n_estimators': n_estimators}
loo = LeaveOneOut()
rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=loo, n_jobs=-1)

grid_search.fit(spec1C_intensity, Y_train)
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (con LOO): {grid_search.best_score_}")


Mejores hiperparámetros: {'n_estimators': 100}
Mejor puntuación (con LOO): 0.7666666666666667


## Test

In [18]:

accuracy_results = {}

for n in n_estimators:
    rf_model = RandomForestClassifier(n_estimators=n, random_state=42)
    y_pred = []
    
    for i in range(len(spec1C_intensity)):
        X_train = np.delete(spec1C_intensity, i, axis=0)
        y_train = np.delete(Y_train, i)
        X_test = specTest_intensity[i].reshape(1, -1)
        rf_model.fit(X_train, y_train)
        pred = rf_model.predict(X_test)
        
        y_pred.append(pred[0])
    
    accuracy = accuracy_score(Y_train, y_pred)
    
    accuracy_results[n] = accuracy

best_n_estimators = max(accuracy_results, key=accuracy_results.get)
print(f" un accuracy de {accuracy_results[best_n_estimators]:.4f}")


 un accuracy de 0.5667


# SVM + RBF

In [19]:
gamma = np.logspace(-3, 3, 10)
C = np.logspace(-3, 3, 10)

In [11]:
k = PairwiseKernel(metric='rbf')
K_train = k( X=spec1C_intensity)
param_grid = {'estimator__C': C, 'estimator__gamma': gamma}
svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed'))
loo = LeaveOneOut()
grid_search = GridSearchCV(estimator=svm_1vsallClos, param_grid=param_grid, cv=loo, n_jobs=-1)

grid_search.fit(K_train, Y_train)
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (con LOO): {grid_search.best_score_}")


Mejores hiperparámetros: {'estimator__C': 1000.0, 'estimator__gamma': 0.001}
Mejor puntuación (con LOO): 0.8333333333333334


In [12]:
C = grid_search.best_params_

## Test

In [20]:
accuracy_results = {}

for g in gamma:
    k = PairwiseKernel(metric='rbf', gamma=g)
    svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed', C=1000))
    y_pred = []
    
    for i in range(len(spec1C_intensity)):
        X_train = np.delete(spec1C_intensity, i, axis=0)
        y_train = np.delete(Y_train, i)
        X_test = specTest_intensity[i].reshape(1, -1)
        K_train = k( X=X_train)
        K_test = k(X=X_train, Y=X_test).T
        svm_1vsallClos.fit(K_train, y_train)
        pred = svm_1vsallClos.predict(K_test)
        
        y_pred.append(pred[0])
    
    accuracy = accuracy_score(Y_train, y_pred)
    
    accuracy_results[n] = accuracy

best_n_estimators = max(accuracy_results, key=accuracy_results.get)
print(f" un accuracy de {accuracy_results[best_n_estimators]:.4f}")


 un accuracy de 0.5667


# SVM + Pike

In [6]:

C = np.logspace(-3, 3, 10)

In [7]:
k = parall.PIKE(t=1)
K_train = k( X_i=spec1C_intensity, X_mz=spec1C_mz)
param_grid = {'estimator__C': C}
svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed'))
loo = LeaveOneOut()
grid_search = GridSearchCV(estimator=svm_1vsallClos, param_grid=param_grid, cv=loo, n_jobs=-1)

grid_search.fit(K_train, Y_train)
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (con LOO): {grid_search.best_score_}")


Mejores hiperparámetros: {'estimator__C': 46.41588833612773}
Mejor puntuación (con LOO): 0.7


In [13]:
C = grid_search.best_params_['estimator__C']

## Others

In [15]:



k = parall.PIKE(t=1)
svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed', C=int(C)))
y_pred = []

for i in range(len(spec1C_intensity)):
    X_train = np.delete(spec1C_intensity, i, axis=0)
    X_train_mz = np.delete(spec1C_mz, i, axis=0)
    y_train = np.delete(Y_train, i)
    X_test = specTest_intensity[i].reshape(1, -1)
    X_test_mz = specTest_mz[i].reshape(1, -1)
    K_train = k( X_i=X_train, X_mz=X_train_mz)
    K_test = k(X_i=X_train, X_mz=X_train_mz, Y_i=X_test, Y_mz=specTest_mz).T
    svm_1vsallClos.fit(K_train, y_train)
    pred = svm_1vsallClos.predict(K_test)
    
    y_pred.append(pred[0])

accuracy = accuracy_score(Y_train, y_pred)


print(f" un accuracy de {accuracy:.4f}")


 un accuracy de 0.4833


# SVM + Linear

In [21]:

C = np.logspace(-3, 3, 10)

In [30]:
k = PairwiseKernel(metric='linear')
K_train = k( X=spec1C_intensity)
param_grid = {'estimator__C': C}
svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed'))
loo = LeaveOneOut()
grid_search = GridSearchCV(estimator=svm_1vsallClos, param_grid=param_grid, cv=loo, n_jobs=-1)

grid_search.fit(K_train, Y_train)
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (con LOO): {grid_search.best_score_}")


Mejores hiperparámetros: {'estimator__C': 1000.0, 'estimator__gamma': 0.001}
Mejor puntuación (con LOO): 0.7833333333333333


In [31]:
C = grid_search.best_params_

## Test

In [22]:
accuracy_results = {}


k = PairwiseKernel(metric='linear')
svm_1vsallClos = OneVsRestClassifier(SVC(kernel='precomputed', C=1000))
y_pred = []

for i in range(len(spec1C_intensity)):
    X_train = np.delete(spec1C_intensity, i, axis=0)
    y_train = np.delete(Y_train, i)
    X_test = specTest_intensity[i].reshape(1, -1)
    K_train = k( X=X_train)
    K_test = k(X=X_train, Y=X_test).T
    svm_1vsallClos.fit(K_train, y_train)
    pred = svm_1vsallClos.predict(K_test)
    
    y_pred.append(pred[0])

accuracy = accuracy_score(Y_train, y_pred)

print(f" un accuracy de {accuracy:.4f}")


 un accuracy de 0.5333


# KNN

In [23]:
n_neighbors = [3, 5, 7, 9, 11]

In [34]:
loo = LeaveOneOut()
knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': n_neighbors,
    
}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=loo, n_jobs=-1)

grid_search.fit(spec1C_intensity, Y_train)
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor puntuación (con LOO): {grid_search.best_score_}")


Mejores hiperparámetros: {'n_neighbors': 3}
Mejor puntuación (con LOO): 0.7166666666666667


## Test

In [24]:
accuracy_results = {}

for n in n_neighbors:
    knn = KNeighborsClassifier(n_neighbors=n)
    y_pred = []
    
    for i in range(len(spec1C_intensity)):
        X_train = np.delete(spec1C_intensity, i, axis=0)
        y_train = np.delete(Y_train, i)
        X_test = specTest_intensity[i].reshape(1, -1)
        knn.fit(X_train, y_train)
        pred = knn.predict(X_test)
        
        y_pred.append(pred[0])
    
    accuracy = accuracy_score(Y_train, y_pred)
    
    accuracy_results[n] = accuracy

best_n_estimators = max(accuracy_results, key=accuracy_results.get)
print(f" un accuracy de {accuracy_results[best_n_estimators]:.4f}")


 un accuracy de 0.4833


# LGBM

In [25]:
Y_train = np.array(Y_train)
Y_train[Y_train=='RT023']=0
Y_train[Y_train=='RT027']=1
Y_train[Y_train=='RT078']=2
Y_train[Y_train=='RT106']=3
Y_train[Y_train=='RT165']=4
Y_train[Y_train=='RT181']=5

In [48]:
param = {'num_leaves': 31, 'objective': 'multiclass', 'num_class': 6}
train_data = lgb.Dataset(spec1C_intensity, label=Y_train)
bst = lgb.train(param, train_data)
ypred = bst.predict(spec1C_intensity, num_iteration=bst.best_iteration)
y_pred = np.argmax(ypred, axis=1)
Y_train_int = Y_train.astype(int)

accuracy = accuracy_score(Y_train_int, y_pred)
print('accuracy: ', accuracy)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 115910
[LightGBM] [Info] Number of data points in the train set: 60, number of used features: 6000
[LightGBM] [Info] Start training from score -1.897120
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.696449
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 

## Test

In [26]:


accuracy_results = {}

param = {'num_leaves': 31, 'objective': 'multiclass', 'num_class': 6}
y_pred = []

for i in range(len(spec1C_intensity)):
    X_train = np.delete(spec1C_intensity, i, axis=0)
    y_train = np.delete(Y_train, i)
    X_test = specTest_intensity[i].reshape(1, -1)
    
    train_data = lgb.Dataset(X_train, label=y_train)
    bst = lgb.train(param, train_data)
    pred = bst.predict(X_test, num_iteration=bst.best_iteration)
    y_pred.append(np.argmax(pred[0])) 

Y_train_int = Y_train.astype(int)

accuracy = accuracy_score(Y_train_int, y_pred)
print(f"Un accuracy de {accuracy:.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017855 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 114694
[LightGBM] [Info] Number of data points in the train set: 59, number of used features: 6000
[LightGBM] [Info] Start training from score -1.880313
[LightGBM] [Info] Start training from score -1.774952
[LightGBM] [Info] Start training from score -1.774952
[LightGBM] [Info] Start training from score -1.774952
[LightGBM] [Info] Start training from score -1.774952
[LightGBM] [Info] Start training from score -1.774952
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] 